In [1]:
# Get pandas for dealing with xlsx
import pandas as pd
# Old numpy
import numpy as np
# Stats
import statistics as st
import statsmodels.api as sm
# Plotting
import matplotlib.pyplot as plt
import math

In [2]:
# set path
path = "/Users/chaseabram/UChiGit/Skwad/Empirical Analysis III/Problem Sets/Heckman Pset 4/PS4_Data.xlsx"
# read file
xls = pd.ExcelFile(path)

In [4]:
# Get number of sheets (4)
num_sheets = len(xls.sheet_names)
# initialize data frame
dfs = [0]*num_sheets

# results data frame
column_names = ['sample', 'mu_1', 'mu_2', 'sigma_1', 'sigma_2', 'sigma_e', 'rho', 'beta_0', 'beta_1', 'beta_2',
               't_stat', 't > 1.964', ]
results = pd.DataFrame(columns = column_names)

# read each sheet into data frame and perform analysis
for i in range(0,num_sheets):
    # Add to data frame
    sheet_name = 'DataSet' + str(i)
    dfs[i] = pd.read_excel(xls, sheet_name)
    
    # read in data
    Y = dfs[i]['Y']
    
    X = np.zeros((3, len(dfs[i])))
    X[0] = [1]*len(dfs[i])
    X[1] = dfs[i]['X1']
    X[2] = dfs[i]['X2']
    
    Z = np.zeros((3, len(dfs[i])))
    Z[0] = [1]*len(dfs[i])
    Z[1] = dfs[i]['Z1']
    Z[2] = dfs[i]['Z2']
    
    
    # means
    mu_1 = np.mean(X[1])
    mu_2 = np.mean(X[2])
    
    # var-cov
    Sigma = np.cov(X)
    sigma_1 = math.sqrt(Sigma[1,1])
    sigma_2 = math.sqrt(Sigma[2,2])
                       
    # rho
    rho = Sigma[2,1]/(Sigma[1,1]*Sigma[2,2])
    
    # run ols with X1 and X2
    model = sm.OLS(Y,X.T)
    result = model.fit()
    betas = result.params
    
    # est var of residuals
    sigma_e = np.cov(result.resid)
    
    # t stat
    tstat = result.tvalues[2]
    
    # t default is reject beta_2 = 0
    t_test = 1
    
    # t test for beta 2
    if abs(tstat) < 1.964:
        # t test cannot reject null of beta_2 = 0
        t_test = 0
        
        # OVB regression replaces beta_0 and beta_1
        model = sm.OLS(Y,X[0:2].T)
        result = model.fit()
        betas[0:2] = result.params 
    
    # create new data frame entry with variables
    new_df = pd.DataFrame(np.array([i, mu_1, mu_2, sigma_1, sigma_2, sigma_e,
                          rho, betas[0], betas[1], betas[2], tstat, t_test])).T
    new_df.columns = column_names
    
    # add to results data frame
    results = pd.concat([results, new_df])

# save estimates
results.to_csv ("/Users/chaseabram/UChiGit/Skwad/Empirical Analysis III/Problem Sets/Heckman Pset 1/Q5_output.csv", index = False, header=True)
results